In [59]:
%reload_ext ishbook
import pandas as pd
import numpy as np
import plus
import datetime as dt
from datetime import datetime
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [60]:
rep_info = plus.read_google_sheet('1dTuz8gPQ8d1REKJ2wwoif34QczcfhkzDOhaW6ubkEJc', intype='key', header=True, sheet='Sales Roster')[[ 'New Team Names', 'User ID']]

#Imported data from Google Sheet with Sales Team reps

In [61]:
NYC_DE1 = rep_info["New Team Names"] == "NYC-DE1"
NYC_DE8 = rep_info["New Team Names"] == "NYC-DE8"

nyc_teams = rep_info[NYC_DE1 | NYC_DE8]

output = None

In [42]:
nyc_teams

,New Team Names,User ID
ID,,
434,NYC-DE1,434
435,NYC-DE1,435
472,NYC-DE1,472
485,NYC-DE1,485
580,NYC-DE1,580
738,NYC-DE1,738
777,NYC-DE1,777
914,NYC-DE1,914
972,NYC-DE1,972


In [62]:
nyc_ids = ",".join(str(x) for x in nyc_teams['User ID'])

#fed the usernames into a string for the NYC-DE1 team

In [63]:
nyc_ids = nyc_ids.rstrip(',')
#removed the comma after the last username

nyc_ids

'434,435,472,485,580,738,777,914,972,1009,1034,1045,1068,1108,1309,1153,1376,1998,2038,2042,1366,1750,2275,2310,2311,2322,2980,2986,2994,2681,2724,3065,3066,3208,3244,3257,3264,3316,3502,3820,1849,4778,4902,5416,5399,3080,4191,3079,5872,3285,5952'

In [64]:
%%ish

callsinbound = from crmcalls yesterday today WHERE rep_id IN ({nyc_ids}) direction IN ('Inbound') talk_time_seconds >= 90 certainty != "uncertain (multiple contacts, no note in 30 minutes)" and client_type = advertiser GROUP BY rep_id/*repid*/, client_id/*advid*/, direction, unixtime SELECT count()/*calls*/
callsoutbound = from crmcalls yesterday today WHERE rep_id IN ({nyc_ids}) direction IN ('Outbound') talk_time_seconds >= 180 certainty != "uncertain (multiple contacts, no note in 30 minutes)" and client_type = advertiser GROUP BY rep_id/*repid*/, client_id/*advid*/, direction, unixtime SELECT count()/*calls*/
emails = from crmemails yesterday today WHERE user_id IN ({nyc_ids}) direction IN ('OUTGOING') and advid > 0 GROUP BY user_id/*repid*/, advid, unixtime SELECT count()/*emails*/

output = None

#Pulls previous day interactions for NYC-DE1/NYC-DE8 teams based on the established criteria

In [65]:
interactions = emails.append(callsinbound).append(callsoutbound)
print len(interactions)
print interactions.head(1)

200
     advid  calls direction  emails  repid    unixtime
0  2321953    NaN       NaN     1.0   2042  1474546211


In [66]:
interactions['Timestamp'] = pd.to_datetime(interactions.unixtime,unit='s') + pd.Timedelta(hours=-5)

In [67]:
repname = plus.get_user_info(interactions.repid.unique())[['firstname','lastname']]
repname.reset_index(inplace=True)
repname['Rep'] = repname.firstname + " " + repname.lastname
repname.head(10)

,index,firstname,lastname,Rep
0,2042,Seth,Bishop,Seth Bishop
1,2681,Alexandra,Tuazon,Alexandra Tuazon
2,1750,John,Akey,John Akey
3,5416,D'ahna,Bishop,D'ahna Bishop
4,3066,Anna,Rofsky,Anna Rofsky
5,3820,Zachary,Carlson,Zachary Carlson
6,1849,Adam,Schankman,Adam Schankman
7,3079,Ryan,Sessa,Ryan Sessa
8,3208,Scott,Johnson,Scott Johnson
9,3316,Pavel,Brutman,Pavel Brutman


In [68]:
advemail = plus.get_advertiser_info(interactions.advid.unique())[['account_email','billing_country','locale','showHostedJobs','active']]
advemail.reset_index(inplace=True)

In [69]:
dirty_merge = interactions.merge(repname,how='inner',left_on='repid',right_on='index').merge(advemail,how='inner',left_on='advid',right_on='index')
len(dirty_merge)

200

In [70]:
dirty_merge.head()

,advid,calls,direction,emails,repid,unixtime,Timestamp,index_x,firstname,lastname,Rep,index_y,account_email,billing_country,locale,showHostedJobs,active
0,2321953,NaN,NaN,1.0,2042,1474546211,2016-09-22 07:10:11,2042,Seth,Bishop,Seth Bishop,2321953,cmccarthy@srcare.org,US,en,verified,1
1,2321953,NaN,NaN,1.0,2042,1474546368,2016-09-22 07:12:48,2042,Seth,Bishop,Seth Bishop,2321953,cmccarthy@srcare.org,US,en,verified,1
2,2321953,NaN,NaN,1.0,2042,1474546416,2016-09-22 07:13:36,2042,Seth,Bishop,Seth Bishop,2321953,cmccarthy@srcare.org,US,en,verified,1
3,2321953,NaN,NaN,1.0,2042,1474553446,2016-09-22 09:10:46,2042,Seth,Bishop,Seth Bishop,2321953,cmccarthy@srcare.org,US,en,verified,1
4,5971228,NaN,NaN,1.0,2042,1474547746,2016-09-22 07:35:46,2042,Seth,Bishop,Seth Bishop,5971228,mlibutti@avenue.com,US,en,enabled,1


In [71]:
dirty_merge = dirty_merge[dirty_merge.showHostedJobs.isin(['verified_spononly','enabled','verified'])]
dirty_merge = dirty_merge[dirty_merge.active.isin(["1","2"])]
dirty_merge = dirty_merge[dirty_merge.locale.isin(['en','de','nl','fr','pt','es','it'])]
dirty_merge = dirty_merge[dirty_merge.billing_country.isin(['US','CA','FR','IT','DE','IE','GB','BR','PT','NL'])]
dirty_merge.tail(3)

,advid,calls,direction,emails,repid,unixtime,Timestamp,index_x,firstname,lastname,Rep,index_y,account_email,billing_country,locale,showHostedJobs,active
197,3540319,NaN,NaN,1.0,2986,1474567067,2016-09-22 12:57:47,2986,John,Prioli,John Prioli,3540319,khayes@cherokeermc.org,US,en,verified,1
198,619674,NaN,NaN,1.0,2986,1474570010,2016-09-22 13:46:50,2986,John,Prioli,John Prioli,619674,paulgrovemanor@comcast.net,US,en,verified,1
199,8963844,1.0,Outbound,NaN,2986,1474551829,2016-09-22 08:43:49,2986,John,Prioli,John Prioli,8963844,hope@rrws.com,US,en,verified,1


In [72]:
dirty_merge.sort_values(['advid','Timestamp'],ascending=True,inplace=True)
no_dupes = dirty_merge.drop_duplicates('advid',keep='last').copy()
len(no_dupes)

154

In [73]:
def interactiontype(x):
    if x['calls'] == 1:
        if x['direction'] == 'Inbound':
            return 'Inbound Call'
        elif x['direction'] == 'Outbound':
            return 'Outbound Call'
        
    elif x['emails'] == 1:
        return 'Email'

    
no_dupes['Interaction Type'] = no_dupes.apply(interactiontype,axis=1)

In [74]:
bill_US = no_dupes["billing_country"] == 'US'
locale_en = no_dupes["locale"] == "en"

bill_BR = no_dupes["billing_country"] == 'BR'
locale_pt = no_dupes["locale"] == "pt"

bill_CA = no_dupes["billing_country"] == 'CA'
locale_fr = no_dupes["locale"] == "fr"

In [75]:
no_dupes.ix[bill_US & locale_en, "locale"] = "us"
no_dupes.ix[bill_BR & locale_pt, "locale"] = "br"
no_dupes.ix[bill_CA & locale_en, "locale"] = "ca"
no_dupes = no_dupes[~(bill_CA & locale_fr)]

In [76]:
mailmerge = no_dupes[['Rep','Interaction Type','account_email','billing_country','locale','Timestamp']].copy()
mailmerge.rename(columns = {"account_email" : "Email"}, inplace = True)
mailmerge.tail()

,Rep,Interaction Type,Email,billing_country,locale,Timestamp
167,Kirby Peck,Email,Kimberly.Soublet@iverify.us,US,us,2016-09-22 13:59:00
49,John Akey,Email,disabledsarahpeck@sigretail.com,US,us,2016-09-22 07:27:43
7,Seth Bishop,Email,JOELSMITH@SPITZER.COM,US,us,2016-09-22 08:05:48
78,Zachary Carlson,Email,ALFry@covenantcareathome.org,US,us,2016-09-22 12:48:42
196,Kelli Paterno,Email,sfarwell@qualitytruck.net,US,us,2016-09-22 16:09:34


In [77]:
print mailmerge.locale.value_counts()

us    154
Name: locale, dtype: int64


In [19]:
csat_sendList = pd.read_csv('8_18_2016_csat.csv')
sales_sentList = pd.read_csv('Sales_tNPS_sentlist.csv')
#if it's the first time running the notebook, comment out the line above, since 'SalesNPSpilotsentlist.csv' has not been created yet


final_output = mailmerge[~mailmerge['Email'].isin(csat_sendList.Email)].copy()
final_output = final_output[~final_output['Email'].isin(sales_sentList.Email)].copy()
#if it's the first time running the notebook, comment out the line above, since 'SalesNPSpilotsentlist.csv' has not been created yet

print len(final_output)
print len(csat_sendList) + len(final_output)

0
166100


In [78]:
date = dt.date.today().strftime("%m%d%Y")
filename = "%s_%s.%s" % ("SalesNPSPilot", date, "csv")
mailmerge.to_csv(filename, index = False)
#final_output.to_csv(filename, index = False)

In [21]:
sales_sendList = final_output["Email"].to_frame()
sales_sentList = sales_sentList.append(sales_sendList, ignore_index = True)
#if it's the first time running the notebook, comment out the line above, since 'SalesNPSpilotsentlist.csv' has not been created yet

print len(sales_sentList)
sales_sentList.to_csv('SalestNPSpilotsentlist.csv', index = False)

132
